In [1]:
import pandas as pd
import duckdb

In [2]:
conn = duckdb.connect("poke_api_data.duckdb")

### Create a more complete pokemon database with addition of types and region informations

In [3]:
query = """
  select 
    pokemon.id as ID,
    pokemon.name as POKEMON,
    types.Type_1 as MAIN_TYPE,
    types.Type_2 as SEC_TYPE,
    regions.main_region as REGION
  from pokemon 
  join types on pokemon.id = types.id
  join regions on pokemon.id = regions.id
"""

df_silver = conn.execute(query).fetchdf()

# Create DuckDB database
conn = duckdb.connect("poke_api_data.duckdb")
conn.execute("""
    CREATE TABLE IF NOT EXISTS s_pokemon_data ( 
      ID INT,
      POKEMON TEXT,
      MAIN_TYPE TEXT,
      SEC_TYPE TEXT,
      REGION TEXT
    )
""")
conn.execute("""INSERT INTO s_pokemon_data SELECT * FROM df_silver""")

### Normalize stats informations

In [4]:
pivt_query = """
SELECT
    id,
    MAX(CASE WHEN stat = 'attack' THEN base_stat END) AS attack,
    MAX(CASE WHEN stat = 'defense' THEN base_stat END) AS defense,
    MAX(CASE WHEN stat = 'speed' THEN base_stat END) AS speed,
    MAX(CASE WHEN stat = 'special-attack' THEN base_stat END) AS sp_atck,
    MAX(CASE WHEN stat = 'special-defense' THEN base_stat END) AS sp_def,
    MAX(CASE WHEN stat = 'hp' THEN base_stat END) AS hp
FROM stats
GROUP BY id
ORDER BY id asc;
"""

stats_pvt = conn.execute(pivt_query).fetch_df()

conn = duckdb.connect("poke_api_data.duckdb")
conn.execute("""
    CREATE TABLE IF NOT EXISTS s_stats ( 
      ID INT,
      ATTACK TEXT,
      DEFENSE TEXT,
      SPEED TEXT,
      SP_ATCK TEXT,
      SP_DEF TEXT,
      HP TEXT
    )
""")
conn.execute("""INSERT INTO s_stats SELECT * FROM stats_pvt""")

In [6]:
conn.close()